In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from scipy.stats import mode
import csv
# import modelvis
from sklearn.ensemble import RandomForestClassifier
%matplotlib

Using matplotlib backend: MacOSX


## Frame 
Given building details, your task is to build a model that can predict the extent of damage that has been done to a building after an earthquake.

## ACQUIRE
train.csv, test.csv, building_ownership_usage.csv, building_structure.csv, sample_submission.csv

In [98]:
df_bd = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/train.csv')
df_bo = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/Building_Ownership_Use.csv')
df_str = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/building_structure.csv')

result = pd.merge(df_str, df_bo, on='building_id')
df = pd.merge(df_bd,result,on="building_id")

In [99]:
df_test = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/test.csv')
df_test = pd.merge(df_test,result,on="building_id")

In [100]:
df.shape
#58 because it has the grade_damage which is predicted data to trrain the classifier

(631761, 58)

In [101]:
df_test.shape

(421175, 57)

## REFINE
Refine the data aquired damage_grade

In [102]:
df.dtypes

area_assesed                               object
building_id                                object
damage_grade                               object
district_id                                 int64
has_geotechnical_risk                     float64
has_geotechnical_risk_fault_crack           int64
has_geotechnical_risk_flood                 int64
has_geotechnical_risk_land_settlement       int64
has_geotechnical_risk_landslide             int64
has_geotechnical_risk_liquefaction          int64
has_geotechnical_risk_other                 int64
has_geotechnical_risk_rock_fall             int64
has_repair_started                        float64
vdcmun_id                                   int64
district_id_x                               int64
vdcmun_id_x                                 int64
ward_id_x                                   int64
count_floors_pre_eq                         int64
count_floors_post_eq                        int64
age_building                                int64


In [103]:
grade = {'Grade 5': 5, 'Grade 4': 4, 'Grade 3': 3, 'Grade 2': 2, 'Grade 1': 1}
area = {'Both': 0, 'Building removed': 1, 'Exterior': 2, 'Not able to inspect': 3, 'Interior': 4}
land = {'Flat': 0, 'Moderate slope': 1, 'Steep slope': 2}
foundation = {'Mud mortar-Stone/Brick': 0, 'Bamboo/Timber': 1, 'Cement-Stone/Brick': 2, 'RC': 3, 'Other': 4}
roof = {'Bamboo/Timber-Light roof':0, 'Bamboo/Timber-Heavy roof':1, 'RCC/RB/RBC': 3}
grnd_floor = {'Mud':0, 'Brick/Stone':1, 'RC':2, 'Timber':3, 'Other':4}
other_floor = {'TImber/Bamboo-Mud': 0, 'Timber-Planck':1, 'Not applicable':2, 'RCC/RB/RBC':3}
pos = {'Not attached':0, 'Attached-1 side':1, 'Attached-2 side':2, 'Attached-3 side':3}
plan = {'Rectangular':0, 'Square':1, 'L-shape':2, 'Multi-projected':3, 'T-shape':4, 'Others':5, 'U-shape':6, 'Building with Central Courtyard':7, 
        'E-shape':8, 'H-shape':9}
condition = {'Damaged-Not used':0, 'Damaged-Repaired and used':1, 'Damaged-Used in risk':2, 'Damaged-Rubble unclear':3, 'Damaged-Rubble clear':4, 
             'Not damaged':5, 'Damaged-Rubble Clear-New building built':6, 'Covered by landslide':7}
legal = {'Private':0, 'Public':1, 'Institutional': 2, 'Other':3}

In [104]:
df.area_assesed = df.area_assesed.map(area.get)
df.land_surface_condition = df.land_surface_condition.map(land.get)
df.foundation_type = df.foundation_type.map(foundation.get)
df.roof_type = df.roof_type.map(roof.get)
df.ground_floor_type = df.ground_floor_type.map(grnd_floor.get)
df.other_floor_type = df.other_floor_type.map(other_floor.get)
df.position = df.position.map(pos.get)
df.plan_configuration = df.plan_configuration.map(plan.get)
df.condition_post_eq = df.condition_post_eq.map(condition.get)
df.legal_ownership_status = df.legal_ownership_status.map(legal.get)

df.damage_grade = df.damage_grade.map(grade.get)

In [109]:
df.isnull().sum()

area_assesed                              0
building_id                               0
damage_grade                              0
district_id                               0
has_geotechnical_risk                     0
has_geotechnical_risk_fault_crack         0
has_geotechnical_risk_flood               0
has_geotechnical_risk_land_settlement     0
has_geotechnical_risk_landslide           0
has_geotechnical_risk_liquefaction        0
has_geotechnical_risk_other               0
has_geotechnical_risk_rock_fall           0
has_repair_started                        0
vdcmun_id                                 0
district_id_x                             0
vdcmun_id_x                               0
ward_id_x                                 0
count_floors_pre_eq                       0
count_floors_post_eq                      0
age_building                              0
plinth_area_sq_ft                         0
height_ft_pre_eq                          0
height_ft_post_eq               

In [106]:
df.count_families.fillna(0, inplace=True)

In [108]:
df.has_repair_started.fillna(1, inplace=True)

In [111]:
df.dtypes

area_assesed                                int64
building_id                                object
damage_grade                                int64
district_id                                 int64
has_geotechnical_risk                     float64
has_geotechnical_risk_fault_crack           int64
has_geotechnical_risk_flood                 int64
has_geotechnical_risk_land_settlement       int64
has_geotechnical_risk_landslide             int64
has_geotechnical_risk_liquefaction          int64
has_geotechnical_risk_other                 int64
has_geotechnical_risk_rock_fall             int64
has_repair_started                        float64
vdcmun_id                                   int64
district_id_x                               int64
vdcmun_id_x                                 int64
ward_id_x                                   int64
count_floors_pre_eq                         int64
count_floors_post_eq                        int64
age_building                                int64


## Exploration

Exploring different and more efficient way to fil the empty value and improve the accuracy

In [12]:
df.has_repair_started.value_counts()

0.0    409222
1.0    189122
Name: has_repair_started, dtype: int64

In [13]:
df.has_repair_started.head(5)

0    0.0
1    1.0
2    0.0
3    0.0
4    0.0
Name: has_repair_started, dtype: float64

#### Lets take has_repair_started as Y and remaining values as X and try predicting the na values

In [14]:
df.columns

Index(['area_assesed', 'building_id', 'damage_grade', 'district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id', 'district_id_x', 'vdcmun_id_x',
       'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_bri

In [110]:
#X_repaired.count_families = X_repaired.count_families.fillna(0, inplace=True)
df.columns

Index(['area_assesed', 'building_id', 'damage_grade', 'district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id', 'district_id_x', 'vdcmun_id_x',
       'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_bri

In [112]:
X = df[['area_assesed', 'damage_grade', 'district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id', 'district_id_x', 'vdcmun_id_x',
       'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'condition_post_eq', 'district_id_y', 'vdcmun_id_y', 'ward_id_y',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']]

y = df.has_repair_started

In [113]:
X.isnull().sum()

area_assesed                              0
damage_grade                              0
district_id                               0
has_geotechnical_risk                     0
has_geotechnical_risk_fault_crack         0
has_geotechnical_risk_flood               0
has_geotechnical_risk_land_settlement     0
has_geotechnical_risk_landslide           0
has_geotechnical_risk_liquefaction        0
has_geotechnical_risk_other               0
has_geotechnical_risk_rock_fall           0
has_repair_started                        0
vdcmun_id                                 0
district_id_x                             0
vdcmun_id_x                               0
ward_id_x                                 0
count_floors_pre_eq                       0
count_floors_post_eq                      0
age_building                              0
plinth_area_sq_ft                         0
height_ft_pre_eq                          0
height_ft_post_eq                         0
land_surface_condition          

In [114]:
model = RandomForestClassifier(max_depth=19, n_estimators=100)
model.fit(X, y)
score = cross_val_score(model, X, y, scoring="accuracy", cv=5, n_jobs=-1)
print(score)
# print(np.mean(score))

[1. 1. 1. 1. 1.]


In [63]:
param = {'max_depth': np.arange(2,20), 'n_estimators': [50, 70, 100]}
clf = GridSearchCV(model, param, scoring='accuracy', return_train_score=True)
clf.fit(X, y)
clf.best_params_

{'max_depth': 19, 'n_estimators': 100}

In [20]:
X_repair_null = df[df['has_repair_started'].isnull()]

In [21]:
X_repair_null.columns

Index(['area_assesed', 'building_id', 'damage_grade', 'district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id', 'district_id_x', 'vdcmun_id_x',
       'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_bri

In [22]:
preX = X_repair_null[['area_assesed', 'district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'vdcmun_id', 'district_id_x', 'vdcmun_id_x',
       'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'condition_post_eq', 'district_id_y', 'vdcmun_id_y', 'ward_id_y',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']]

In [23]:
out = model.predict(preX)

In [27]:
X_repair_null.has_repair_started = out

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [29]:
X_repair_null.has_repair_started.head(5)

8      0.0
57     1.0
63     1.0
89     0.0
100    0.0
Name: has_repair_started, dtype: float64

In [34]:
X_repair_null.isnull().sum()

area_assesed                              0
building_id                               0
damage_grade                              0
district_id                               0
has_geotechnical_risk                     0
has_geotechnical_risk_fault_crack         0
has_geotechnical_risk_flood               0
has_geotechnical_risk_land_settlement     0
has_geotechnical_risk_landslide           0
has_geotechnical_risk_liquefaction        0
has_geotechnical_risk_other               0
has_geotechnical_risk_rock_fall           0
has_repair_started                        0
vdcmun_id                                 0
district_id_x                             0
vdcmun_id_x                               0
ward_id_x                                 0
count_floors_pre_eq                       0
count_floors_post_eq                      0
age_building                              0
plinth_area_sq_ft                         0
height_ft_pre_eq                          0
height_ft_post_eq               

In [35]:
df1 = X_repair_null.copy()

In [36]:
df1.head(5)

,area_assesed,building_id,damage_grade,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
8,0,1e96b58fa58,1,21,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57,0,1c3d6a2f185e,1,31,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63,0,2d456a90fd2,1,31,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,0,2d6ad13c9cf,1,31,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,0,2da2bbe2cef,1,31,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [115]:
df.building_id = df.building_id.apply(lambda x: int(x, 16))

In [133]:
df.head(1)

,area_assesed,building_id,damage_grade,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,0,2489030005410,4,24,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Model

In [124]:
df_test.columns

Index(['area_assesed', 'building_id', 'district_id', 'has_geotechnical_risk',
       'has_geotechnical_risk_fault_crack', 'has_geotechnical_risk_flood',
       'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id', 'district_id_x', 'vdcmun_id_x',
       'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_supers

In [122]:
df_test.has_repair_started.fillna(1, inplace=True)

In [131]:
df_test.building_id = df_test.building_id.apply(lambda x: int(x, 16))

In [119]:
df_test.area_assesed = df_test.area_assesed.map(area.get)
df_test.land_surface_condition = df_test.land_surface_condition.map(land.get)
df_test.foundation_type = df_test.foundation_type.map(foundation.get)
df_test.roof_type = df_test.roof_type.map(roof.get)
df_test.ground_floor_type = df_test.ground_floor_type.map(grnd_floor.get)
df_test.other_floor_type = df_test.other_floor_type.map(other_floor.get)
df_test.position = df_test.position.map(pos.get)
df_test.plan_configuration = df_test.plan_configuration.map(plan.get)
df_test.condition_post_eq = df_test.condition_post_eq.map(condition.get)
df_test.legal_ownership_status = df_test.legal_ownership_status.map(legal.get)

In [65]:
model_train = RandomForestClassifier(max_depth=19, n_estimators=100)
model_train.fit(X_train, y_train)
score = cross_val_score(model, X_train, y_train, scoring="accuracy", cv=5, n_jobs=-1)
print(score)

[0.9895288  0.99012567 0.99072412 0.9905731  0.98997156]


## Test

In [72]:
df_test.area_assesed = df_test.area_assesed.map(area.get)
df_test.land_surface_condition = df_test.land_surface_condition.map(land.get)
df_test.foundation_type = df_test.foundation_type.map(foundation.get)
df_test.roof_type = df_test.roof_type.map(roof.get)
df_test.ground_floor_type = df_test.ground_floor_type.map(grnd_floor.get)
df_test.other_floor_type = df_test.other_floor_type.map(other_floor.get)
df_test.position = df_test.position.map(pos.get)
df_test.plan_configuration = df_test.plan_configuration.map(plan.get)
df_test.condition_post_eq = df_test.condition_post_eq.map(condition.get)
df_test.legal_ownership_status = df_test.legal_ownership_status.map(legal.get)

In [126]:
df_test.isnull().sum()

area_assesed                              0
building_id                               0
district_id                               0
has_geotechnical_risk                     0
has_geotechnical_risk_fault_crack         0
has_geotechnical_risk_flood               0
has_geotechnical_risk_land_settlement     0
has_geotechnical_risk_landslide           0
has_geotechnical_risk_liquefaction        0
has_geotechnical_risk_other               0
has_geotechnical_risk_rock_fall           0
has_repair_started                        0
vdcmun_id                                 0
district_id_x                             0
vdcmun_id_x                               0
ward_id_x                                 0
count_floors_pre_eq                       0
count_floors_post_eq                      0
age_building                              0
plinth_area_sq_ft                         0
height_ft_pre_eq                          0
height_ft_post_eq                         0
land_surface_condition          

In [127]:
df_test.columns

Index(['area_assesed', 'building_id', 'district_id', 'has_geotechnical_risk',
       'has_geotechnical_risk_fault_crack', 'has_geotechnical_risk_flood',
       'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id', 'district_id_x', 'vdcmun_id_x',
       'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_supers

In [128]:
df1 = df_test[['area_assesed', 'district_id', 'has_geotechnical_risk',
       'has_geotechnical_risk_fault_crack', 'has_geotechnical_risk_flood',
       'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id', 'district_id_x', 'vdcmun_id_x',
       'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'condition_post_eq', 'district_id_y', 'vdcmun_id_y', 'ward_id_y',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']]

In [132]:
try:
    with open('output_repairLogic.csv', 'w') as csvfp:
        wr = csv.writer(csvfp)
        for i in range(len(df1)):
#             print([format(df.building_id[i], 'x'), 'Grade '+str(model_train.predict([df1.iloc[i]])[0])])
            wr.writerow([format(df_test.building_id[i], 'x'), 'Grade '+str(model_train.predict([df1.iloc[i]])[0])])
#             break
except Exception as ex:
    print(ex)
finally:
    csvfp.close()